# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS-109A Introduction to Data Science 


## Fine Particulate Air Pollution and COVID-19

**Harvard University**<br>
**Spring 2020**<br>
Jack Luby, Hakeem Angulu, and Louie Ayre <br>

---



### The Problem

Fine particulate matter (PM$_{2.5}$) is an air pollutant which has been shown to increase the risk of mortality and hospitalization in exposed populations. 

The fine inhalable particles of PM$_{2.5}$ impact communitites at the local level, incrementally decreasing their life expectancies as ambient concentrations of the pollutant rise. Despite their adverse effects, concentrations of PM$_{2.5}$ are not well monitored throughout much of the United States (especially in regions of low population density). As a result, many communities (and their care providers) are unaware of the life-shortening ambient concentrations of PM$_{2.5}$ they breathe each day. 

These risks have been heightened by the coronavirus pandemic. COVID-19 is primarily a respiratory disease, and PM$_{2.5}$'s adverse effects on respiratory potential have been theorized and shown to increase the likelihood of developing COVID-19. It is apparent that high concentrations of PM$_{2.5}$ are likely to be founded in rural and low-income communities and communities of color. In addition, the US healthcare and social systems have long underserved those communities. The combination of these factors creates an especially bad prognosis for those communities, and necessitates further study and rapid policy and healthcare interventions.

### Our Project

With PM$_{2.5}$ data and COVID-19 data at hand, this project seeks to understand the relationships between PM$_{2.5}$ pollution, demographic and socioeconomic factors, and COVID-19, with a particular focus on rural and low-income communities, and communities of color.

In [1]:
## Set formatting to CS109 standard
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
# The classics
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

## Import and Prepare Data

### PM_25 Data

These data were obtained from the __Exposure to air pollution and COVID-19 mortality in the United States__ data repository, found [here](https://github.com/wxwx1993/PM_COVID).

In [14]:
# Import sample airpred data
no_loc_airpred = pd.read_csv("data/airpred.csv")
locations = pd.read_csv("data/airpred_monitor_locations.csv")

In [13]:
# Merge location data
airpred = no_loc_airpred.merge(locations, how='left')

In [28]:
# County PM data
init_county_pm = pd.read_csv("data/county_pm25.csv")

In [29]:
init_county_pm.head()

,fips,year,pm25
0,36103.0,2000,13.749745
1,36103.0,2001,13.681471
2,36103.0,2002,12.549986
3,36103.0,2003,12.436192
4,36103.0,2004,11.738100


In [33]:
# Drop the counties with no data and no fips
county_pm = init_county_pm.dropna()

In [42]:
# Convert fips to int
county_pm['fips'] = county_pm['fips'].astype(int)

In [35]:
county_pm.head()

,fips,year,pm25
0,36103,2000,13.749745
1,36103,2001,13.681471
2,36103,2002,12.549986
3,36103,2003,12.436192
4,36103,2004,11.738100


### Socioeconomic, Demographic, and Behavioral Risk Factor Data

These data were obtained from the __Exposure to air pollution and COVID-19 mortality in the United States__ data repository, found [here](https://github.com/wxwx1993/PM_COVID).

In [36]:
# County socioeconomic and demographic data
init_county_demo = pd.read_csv("data/census_county_interpolated.csv")

In [39]:
# Drop unnecessary axis
init_county_demo.drop(['Unnamed: 0'], axis=1, inplace=True)

In [41]:
init_county_demo.head()

,fips,year,poverty,popdensity,medianhousevalue,pct_blk,medhouseholdincome,pct_owner_occ,hispanic,education,population,pct_asian,pct_native,pct_white
0,36103.0,2000,0.058031,1875.609065,239247.243803,0.047576,66265.088050,0.808135,0.072071,0.246606,1.450411e+06,0.018570,0.002206,0.893031
1,36103.0,2004,0.056872,1883.031226,269377.542545,0.047693,68979.083857,0.811235,0.075708,0.234804,1.457034e+06,0.019281,0.002208,0.892249
2,36103.0,2008,0.047349,1942.548304,506122.762671,0.048704,90873.962264,0.833869,0.105376,0.135350,1.511674e+06,0.025127,0.002184,0.886210
3,36103.0,2011,0.047716,1945.335549,510830.523810,0.048919,90295.504762,0.835889,0.104793,0.139874,1.494680e+06,0.024687,0.002195,0.885766
4,36103.0,2014,0.049384,1956.409276,481809.390476,0.049218,92309.723810,0.824198,0.110059,0.120635,1.508614e+06,0.027491,0.002044,0.880521


In [44]:
# Drop the counties with no fips
county_demo = init_county_demo.dropna()

In [45]:
# Convert fips to int
county_demo['fips'] = county_demo['fips'].astype(int)

/Users/hakeemangulu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [46]:
county_demo.head()

,fips,year,poverty,popdensity,medianhousevalue,pct_blk,medhouseholdincome,pct_owner_occ,hispanic,education,population,pct_asian,pct_native,pct_white
0,36103,2000,0.058031,1875.609065,239247.243803,0.047576,66265.088050,0.808135,0.072071,0.246606,1.450411e+06,0.018570,0.002206,0.893031
1,36103,2004,0.056872,1883.031226,269377.542545,0.047693,68979.083857,0.811235,0.075708,0.234804,1.457034e+06,0.019281,0.002208,0.892249
2,36103,2008,0.047349,1942.548304,506122.762671,0.048704,90873.962264,0.833869,0.105376,0.135350,1.511674e+06,0.025127,0.002184,0.886210
3,36103,2011,0.047716,1945.335549,510830.523810,0.048919,90295.504762,0.835889,0.104793,0.139874,1.494680e+06,0.024687,0.002195,0.885766
4,36103,2014,0.049384,1956.409276,481809.390476,0.049218,92309.723810,0.824198,0.110059,0.120635,1.508614e+06,0.027491,0.002044,0.880521


In [50]:
# County behavioral risk factor data
county_brf = pd.read_csv("data/brfss_county_interpolated.csv")

### COVID-19 Data

These data were obtained from the __COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University__, found [here](https://github.com/CSSEGISandData/COVID-19).

In [67]:
def covid_data_prep(date):
    """
    Process COVID-19 county-wise data.
    
    date (string): the date of the data
    """
    init_county_covid = pd.read_csv(f"data/county_covid_{date}.csv")
    
    # Select columns of interest
    select_county_covid = init_county_covid[['FIPS', 'Confirmed', 'Deaths', 'Recovered', 'Active']]
    
    # Add a date column
    select_county_covid['date'] = date
    
    # Drop NA
    select_county_covid.dropna(inplace=True)
    
    # Convert fips to int and format the columns
    select_county_covid['FIPS'] = select_county_covid['FIPS'].astype(int)
    return select_county_covid.rename(str.lower, axis=1)

In [71]:
# Available dates
available_dates = ['03_25', '04_05', '04_15', '04_25', '05_05']

In [72]:
all_county_covid_data = [covid_data_prep(date) for date in available_dates]

/Users/hakeemangulu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/hakeemangulu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/hakeemangulu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [74]:
all_county_covid_data[-1]

,fips,confirmed,deaths,recovered,active,date
0,45001,33,0,0,33,05_05
1,22001,136,10,0,126,05_05
2,51001,429,7,0,422,05_05
3,16001,713,19,0,694,05_05
4,19001,2,0,0,2,05_05
...,...,...,...,...,...,...
2961,99999,103,3,0,100,05_05
2964,66,145,5,0,140,05_05
2991,69,14,2,0,12,05_05
2997,72,1924,99,0,1825,05_05
